# 6. Решение уравнения теплопроводности методом сеток

In [27]:
from math import cos
from pandas import DataFrame as df

In [9]:
def a(x, t):
    return cos(x)

def b(x, t):
    return x

def c(x, t):
    return 0

alpha1 = 0
alpha2 = -1
beta1 = 1
beta2 = 1

In [33]:
def u(x, t):
    return x**3 + t**3

def dudx(x, t):
    return 3*(x**2)

def dudt(x, t):
    return 3*(t**2)

def d2udx2(x, t):
    return 6 * x

def phi(x):
    return u(x, 0)

def alpha(t):
    return alpha1 * u(0, t) - alpha2 * dudx(0, t)

def beta(t):
    return beta1 * u(1, t) + beta2 * dudx(1, t)

def f(x, t):
    return dudt(x, t) - a(x, t) * d2udx2(x, t) - b(x, t) * dudx(x, t) - c(x, t) * u(x, t)

## Явная разностная схема

In [34]:
# подбирает m из условия устойчивости 5m >= n^2
def calc_m(n):
    m = 5
    while 5 * m < n * n:
        m *= 2
    return m

In [101]:
def solve_explicit(n, m):
    h = 1/n
    tau = 0.1/m
    x = [i * h for i in range(n+1)]
    u = [[0 for i in range(m+1)] for j in range(n+1)]

    for i in range(n+1):
        u[i][0] = phi(x[i])
    
    def L(i, k, h):
        return a(x[i], tau * k) * (u[i + 1][k] - 2 * u[i][k] + u[i - 1][k]) / (h ** 2) + b(x[i], tau * k) * \
               (u[i + 1][k] - u[i - 1][k]) / (2 * h) + c(x[i], tau * k) * u[i][k]
    
    for k in range(1, m+1):
        for i in range(1, n):
            u[i][k] = u[i][k-1] + tau * (L(i, k - 1, h) + f(x[i], tau * (k - 1)))
        u[0][k] = (alpha(tau * k) + alpha2 * (4 * u[1][k] - u[2][k]) / (2 * h)) / (alpha1 + 3 * alpha2 / (2 * h))
        u[n][k] = (beta(tau * k) - beta2 * (-4 * u[n - 1][k] + u[n - 2][k]) / (2 * h)) / (beta1 + 3 * beta2 / (2 * h))
    
    return u                        

In [102]:
N = 5
M = calc_m(5)
N, M

(5, 5)

In [103]:
table = df(data=solve_explicit(N, M), index=[i / N for i in range(N+1)], columns=[i * 0.1 / M for i in range(M+1)])
table

,0.00,0.02,0.04,0.06,0.08,0.10
0.0,0.000,-0.010560,-0.017186,-0.020626,-0.023848,-0.025852
0.2,0.008,0.008160,0.003278,0.000247,-0.002247,-0.003830
0.4,0.064,0.064320,0.064670,0.062866,0.062557,0.062236
0.6,0.216,0.216480,0.216994,0.219175,0.220496,0.222168
0.8,0.512,0.512640,0.516899,0.520311,0.523455,0.526283
1.0,1.000,1.010024,1.014891,1.018281,1.021626,1.024519


In [104]:
N = 10
M = calc_m(10)
N, M

(10, 20)

In [105]:
table = df(data=solve_explicit(N, M), index=[i / N for i in range(N+1)], columns=[i * 0.1 / M for i in range(M+1)])
table

,0.000,0.005,0.010,0.015,0.020,0.025,0.030,0.035,0.040,0.045,...,0.055,0.060,0.065,0.070,0.075,0.080,0.085,0.090,0.095,0.100
0.0,0.000,-0.001330,-0.002204,-0.002674,-0.003122,-0.003468,-0.003796,-0.004073,-0.004333,-0.004560,...,-0.004954,-0.005120,-0.005264,-0.005389,-0.005492,-0.005575,-0.005637,-0.005678,-0.005697,-0.005694
0.1,0.001,0.001005,0.000352,-0.000078,-0.000464,-0.000788,-0.001084,-0.001345,-0.001585,-0.001799,...,-0.002168,-0.002322,-0.002456,-0.002571,-0.002665,-0.002739,-0.002792,-0.002825,-0.002836,-0.002826
0.2,0.008,0.008010,0.008020,0.007713,0.007508,0.007254,0.007052,0.006839,0.006659,0.006483,...,0.006191,0.006072,0.005968,0.005884,0.005817,0.005770,0.005742,0.005733,0.005745,0.005777
0.3,0.027,0.027015,0.027030,0.027047,0.026916,0.026828,0.026688,0.026579,0.026453,0.026352,...,0.026171,0.026099,0.026046,0.026005,0.025984,0.025977,0.025989,0.026018,0.026066,0.026132
0.4,0.064,0.064020,0.064040,0.064062,0.064086,0.064045,0.064020,0.063967,0.063934,0.063892,...,0.063847,0.063842,0.063845,0.063865,0.063895,0.063942,0.064003,0.064080,0.064173,0.064283
0.5,0.125,0.125025,0.125051,0.125077,0.125106,0.125138,0.125155,0.125187,0.125213,0.125252,...,0.125345,0.125404,0.125475,0.125554,0.125646,0.125749,0.125865,0.125994,0.126137,0.126295
0.6,0.216,0.216030,0.216061,0.216092,0.216126,0.216189,0.216268,0.216355,0.216455,0.216558,...,0.216789,0.216916,0.217049,0.217191,0.217340,0.217500,0.217670,0.217850,0.218043,0.218247
0.7,0.343,0.343035,0.343071,0.343107,0.343207,0.343330,0.343475,0.343631,0.343796,0.343967,...,0.344322,0.344506,0.344696,0.344891,0.345092,0.345301,0.345517,0.345742,0.345977,0.346222
0.8,0.512,0.512040,0.512081,0.512275,0.512486,0.512709,0.512934,0.513160,0.513387,0.513614,...,0.514068,0.514297,0.514528,0.514762,0.515001,0.515244,0.515494,0.515751,0.516016,0.516289
0.9,0.729,0.729045,0.729505,0.729880,0.730215,0.730523,0.730813,0.731089,0.731355,0.731615,...,0.732122,0.732374,0.732625,0.732878,0.733134,0.733394,0.733659,0.733930,0.734209,0.734495


In [106]:
N = 20
M = calc_m(20)
N, M

(20, 80)

In [107]:
table = df(data=solve_explicit(N, M), index=[i / N for i in range(N+1)], columns=[i * 0.1 / M for i in range(M+1)])
table

,0.00000,0.00125,0.00250,0.00375,0.00500,0.00625,0.00750,0.00875,0.01000,0.01125,...,0.08875,0.09000,0.09125,0.09250,0.09375,0.09500,0.09625,0.09750,0.09875,0.10000
0.00,0.000000,-0.000167,-0.000277,-0.000337,-0.000395,-0.000440,-0.000484,-0.000522,-0.000559,-0.000591,...,-0.000894,-0.000874,-0.000852,-0.000830,-0.000807,-0.000782,-0.000757,-0.000730,-0.000703,-0.000674
0.05,0.000125,0.000125,0.000042,-0.000013,-0.000063,-0.000106,-0.000146,-0.000182,-0.000216,-0.000247,...,-0.000529,-0.000508,-0.000487,-0.000464,-0.000440,-0.000416,-0.000390,-0.000364,-0.000336,-0.000308
0.10,0.001000,0.001000,0.001001,0.000960,0.000932,0.000898,0.000870,0.000839,0.000812,0.000785,...,0.000568,0.000589,0.000611,0.000634,0.000658,0.000683,0.000709,0.000736,0.000764,0.000793
0.15,0.003375,0.003375,0.003376,0.003376,0.003357,0.003343,0.003322,0.003305,0.003283,0.003265,...,0.003146,0.003168,0.003190,0.003214,0.003238,0.003264,0.003290,0.003318,0.003346,0.003376
0.20,0.008000,0.008001,0.008001,0.008002,0.008003,0.007993,0.007987,0.007975,0.007965,0.007951,...,0.007956,0.007978,0.008002,0.008026,0.008051,0.008078,0.008105,0.008133,0.008163,0.008193
0.25,0.015625,0.015626,0.015627,0.015627,0.015628,0.015629,0.015625,0.015623,0.015616,0.015611,...,0.015748,0.015771,0.015796,0.015821,0.015848,0.015875,0.015903,0.015933,0.015963,0.015994
0.30,0.027000,0.027001,0.027002,0.027003,0.027004,0.027005,0.027006,0.027005,0.027004,0.027001,...,0.027274,0.027299,0.027324,0.027351,0.027379,0.027407,0.027437,0.027467,0.027499,0.027531
0.35,0.042875,0.042876,0.042877,0.042878,0.042879,0.042881,0.042882,0.042883,0.042884,0.042884,...,0.043286,0.043312,0.043339,0.043367,0.043396,0.043426,0.043457,0.043488,0.043521,0.043555
0.40,0.064000,0.064001,0.064003,0.064004,0.064005,0.064006,0.064008,0.064009,0.064011,0.064012,...,0.064536,0.064563,0.064592,0.064622,0.064652,0.064683,0.064715,0.064749,0.064783,0.064818
0.45,0.091125,0.091126,0.091128,0.091129,0.091131,0.091132,0.091134,0.091135,0.091137,0.091139,...,0.091777,0.091806,0.091836,0.091867,0.091899,0.091932,0.091965,0.092000,0.092036,0.092072


## Схема с весами

In [94]:
def solve_weights(n, m):
    h = 1/n
    tau = 0.1/m
    x = [i * h for i in range(n+1)]
    u = [[0 for i in range(m+1)] for j in range(n+1)]
    t = [i * tau for i in range(m+1)]
    sigma = 1/2
    
    for i in range(n + 1):
        u[i][0] = phi(x[i])
        
    def L(i, k, h):
        return a(x[i], tau * k) * (u[i + 1][k] - 2 * u[i][k] + u[i - 1][k]) / (h ** 2) + b(x[i], tau * k) * \
               (u[i + 1][k] - u[i - 1][k]) / (2 * h) + c(x[i], tau * k) * u[i][k]

    for k in range(1, m+1):
        A = [0]
        B = [-alpha1 - alpha2 / h]
        C = [-alpha2 / h]
        G = [alpha(t[k])]
        for i in range(1, n):
            A.append(sigma * a(x[i], t[k]) / (h ** 2) - sigma * b(x[i], t[k]) / (2 * h))
            B.append(2 * sigma * a(x[i], t[k]) / (h ** 2) - sigma * c(x[i], t[k]) + 1 / tau)
            C.append(sigma * a(x[i], t[k]) / (h ** 2) + sigma * b(x[i], t[k]) / (2 * h))
            G.append(-u[i][k - 1] / tau - (1 - sigma) * L(i, k - 1, h) - f(x[i], t[k] - tau / 2))
        A.append(-beta2 / h)
        B.append(-beta1 - beta2 / h)
        C.append(0)
        G.append(beta(tau * k))
        
        S = [0] * len(A)
        T = [0] * len(A)
        
        S[0] = C[0] / B[0]
        for i in range(1, len(A)):
            S[i] = C[i] / (B[i] - A[i] * S[i-1])
    
        T[0] = -G[0] / B[0]
        for i in range(1, len(A)):
            T[i] = (A[i] * T[i-1] - G[i]) / (B[i] - A[i] * S[i-1])
        
        u[len(A) - 1][k] = T[len(A) - 1]
        for i in range(len(A) - 2, -1, -1):
            u[i][k] = S[i] * u[i+1][k] + T[i]
        
    return u

In [95]:
N = 5
M = calc_m(5)
N, M

(5, 5)

In [96]:
table = df(data=solve_weights(N, M), index=[i / N for i in range(N+1)], columns=[i * 0.1 / M for i in range(M+1)])
table

,0.00,0.02,0.04,0.06,0.08,0.10
0.0,0.000,0.009869,0.013397,0.017405,0.022716,0.029554
0.2,0.008,0.009869,0.013397,0.017405,0.022716,0.029554
0.4,0.064,0.065002,0.068159,0.074073,0.082116,0.091561
0.6,0.216,0.219020,0.227793,0.240575,0.254266,0.267987
0.8,0.512,0.528211,0.556173,0.578231,0.596927,0.613489
1.0,1.000,1.106844,1.130155,1.148562,1.164191,1.178074


In [97]:
N = 10
M = calc_m(10)
N, M

(10, 20)

In [98]:
table = df(data=solve_weights(N, M), index=[i / N for i in range(N+1)], columns=[i * 0.1 / M for i in range(M+1)])
table

,0.000,0.005,0.010,0.015,0.020,0.025,0.030,0.035,0.040,0.045,...,0.055,0.060,0.065,0.070,0.075,0.080,0.085,0.090,0.095,0.100
0.0,0.000,0.001211,0.001569,0.001841,0.002073,0.002282,0.002481,0.002676,0.002877,0.003091,...,0.003589,0.003885,0.004220,0.004599,0.005024,0.005499,0.006026,0.006605,0.007237,0.007924
0.1,0.001,0.001211,0.001569,0.001841,0.002073,0.002282,0.002481,0.002676,0.002877,0.003091,...,0.003589,0.003885,0.004220,0.004599,0.005024,0.005499,0.006026,0.006605,0.007237,0.007924
0.2,0.008,0.008045,0.008163,0.008327,0.008496,0.008667,0.008843,0.009032,0.009241,0.009476,...,0.010054,0.010407,0.010806,0.011255,0.011755,0.012308,0.012914,0.013573,0.014286,0.015051
0.3,0.027,0.027021,0.027063,0.027137,0.027241,0.027372,0.027537,0.027742,0.027994,0.028300,...,0.029084,0.029566,0.030107,0.030707,0.031364,0.032076,0.032843,0.033662,0.034533,0.035452
0.4,0.064,0.064021,0.064051,0.064101,0.064191,0.064338,0.064558,0.064859,0.065248,0.065723,...,0.066920,0.067633,0.068414,0.069258,0.070161,0.071118,0.072125,0.073179,0.074277,0.075416
0.5,0.125,0.125027,0.125070,0.125157,0.125332,0.125626,0.126056,0.126623,0.127315,0.128122,...,0.130020,0.131087,0.132218,0.133405,0.134639,0.135916,0.137230,0.138576,0.139953,0.141356
0.6,0.216,0.216042,0.216151,0.216415,0.216901,0.217622,0.218550,0.219649,0.220884,0.222226,...,0.225136,0.226671,0.228244,0.229845,0.231468,0.233108,0.234761,0.236424,0.238096,0.239775
0.7,0.343,0.343114,0.343533,0.344431,0.345767,0.347403,0.349229,0.351169,0.353175,0.355216,...,0.359328,0.361377,0.363413,0.365434,0.367437,0.369422,0.371389,0.373338,0.375270,0.377186
0.8,0.512,0.512576,0.514398,0.517148,0.520165,0.523215,0.526209,0.529118,0.531931,0.534650,...,0.539826,0.542296,0.544696,0.547032,0.549310,0.551535,0.553711,0.555845,0.557939,0.559997
0.9,0.729,0.732900,0.739818,0.745403,0.750173,0.754389,0.758202,0.761705,0.764962,0.768017,...,0.773650,0.776272,0.778789,0.781213,0.783555,0.785826,0.788032,0.790182,0.792282,0.794336


In [99]:
N = 20
M = calc_m(20)
N, M

(20, 80)

In [100]:
table = df(data=solve_weights(N, M), index=[i / N for i in range(N+1)], columns=[i * 0.1 / M for i in range(M+1)])
table

,0.00000,0.00125,0.00250,0.00375,0.00500,0.00625,0.00750,0.00875,0.01000,0.01125,...,0.08875,0.09000,0.09125,0.09250,0.09375,0.09500,0.09625,0.09750,0.09875,0.10000
0.00,0.000000,0.000151,0.000195,0.000229,0.000257,0.000281,0.000303,0.000324,0.000343,0.000361,...,0.002228,0.002300,0.002374,0.002451,0.002529,0.002609,0.002691,0.002776,0.002862,0.002951
0.05,0.000125,0.000151,0.000195,0.000229,0.000257,0.000281,0.000303,0.000324,0.000343,0.000361,...,0.002228,0.002300,0.002374,0.002451,0.002529,0.002609,0.002691,0.002776,0.002862,0.002951
0.10,0.001000,0.001005,0.001019,0.001038,0.001058,0.001077,0.001095,0.001112,0.001129,0.001145,...,0.003061,0.003136,0.003212,0.003291,0.003371,0.003454,0.003539,0.003626,0.003714,0.003805
0.15,0.003375,0.003376,0.003380,0.003388,0.003398,0.003409,0.003422,0.003434,0.003447,0.003459,...,0.005482,0.005561,0.005643,0.005726,0.005811,0.005898,0.005988,0.006079,0.006173,0.006269
0.20,0.008000,0.008001,0.008002,0.008005,0.008009,0.008015,0.008022,0.008030,0.008039,0.008048,...,0.010250,0.010336,0.010424,0.010514,0.010607,0.010701,0.010798,0.010896,0.010997,0.011100
0.25,0.015625,0.015626,0.015627,0.015628,0.015630,0.015633,0.015637,0.015642,0.015647,0.015653,...,0.018127,0.018223,0.018321,0.018421,0.018522,0.018626,0.018732,0.018840,0.018950,0.019062
0.30,0.027000,0.027001,0.027002,0.027003,0.027004,0.027006,0.027008,0.027011,0.027015,0.027018,...,0.029884,0.029991,0.030101,0.030212,0.030326,0.030442,0.030559,0.030679,0.030800,0.030924
0.35,0.042875,0.042876,0.042877,0.042878,0.042880,0.042881,0.042883,0.042885,0.042887,0.042890,...,0.046292,0.046414,0.046537,0.046663,0.046790,0.046920,0.047051,0.047184,0.047319,0.047456
0.40,0.064000,0.064001,0.064003,0.064004,0.064005,0.064007,0.064008,0.064010,0.064012,0.064014,...,0.068131,0.068269,0.068408,0.068550,0.068693,0.068838,0.068985,0.069133,0.069284,0.069436
0.45,0.091125,0.091126,0.091128,0.091129,0.091131,0.091132,0.091134,0.091136,0.091138,0.091140,...,0.096182,0.096337,0.096495,0.096654,0.096815,0.096977,0.097141,0.097307,0.097474,0.097643
